# Activity Clustering for Process Mining

This notebook clusters activities according to the predefined mapping:

| Clustered Activity | Activities in Cluster |
|---|---|
| A_Create Application | A_Create Application, A_Submitted, A_Concept |
| W_Complete application | W_Complete application, A_Accepted, O_Create offer, O_Created, O_Sent, W_Call after offers, A_Complete |
| W_Call incomplete files | W_Call incomplete files, A_Incomplete |
| W_Validate application | W_Validate application, A_Validating, O_Returned |
| A_Denied | A_Denied, O_Refused |
| A_Cancelled | A_Cancelled, O_Cancelled |
| A_Pending | O_Accepted, A_Pending |

For resources: the resource that executes the first activity in the cluster is assumed to execute all activities in that cluster.

In [ ]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('application_log_clean_timestamp.csv') # log with estimated timestamps from Apromore
print(f"Original dataset shape: {df.shape}")
print(f"Unique activities: {df['activity'].nunique()}")
df.head()

Original dataset shape: (1202267, 20)
Unique activities: 26


,Action,resource,activity,EventOrigin,EventID,lifecycle_transition,case_LoanGoal,case_ApplicationType,case_id,case_RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,End_Time,Start_Time
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01T10:51:15.304,2016-01-01T10:51:15.304
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01T10:51:15.352,2016-01-01T10:51:15.304
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01T10:51:15.774,2016-01-01T10:51:15.352
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01T10:52:36.392,2016-01-01T10:51:15.774
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01T10:52:36.403,2016-01-01T10:52:36.392


In [ ]:
# Define the activity clustering mapping
activity_to_cluster = {
    # A_Create Application cluster
    'A_Create Application': 'A_Create Application',
    'A_Submitted': 'A_Create Application',
    # 'A_Concept' removed from cluster - kept as standalone
    
    # W_Complete application cluster
    'W_Complete application': 'W_Complete application',
    'A_Accepted': 'W_Complete application',
    # 'O_Create Offer' removed from cluster - kept as standalone
    'O_Created': 'W_Complete application',
    'O_Sent (mail and online)': 'W_Complete application',
    'O_Sent (online only)': 'W_Complete application',
    'W_Call after offers': 'W_Complete application',
    # 'A_Complete' removed from cluster - kept as standalone
    
    # W_Call incomplete files cluster
    'W_Call incomplete files': 'W_Call incomplete files',
    'A_Incomplete': 'W_Call incomplete files',
    
    # W_Validate application cluster
    'W_Validate application': 'W_Validate application',
    'A_Validating': 'W_Validate application',
    'O_Returned': 'W_Validate application',
    
    # A_Denied cluster
    'A_Denied': 'A_Denied',
    'O_Refused': 'A_Denied',
    
    # A_Cancelled cluster
    'A_Cancelled': 'A_Cancelled',
    'O_Cancelled': 'A_Cancelled',
    
    # A_Pending cluster
    'O_Accepted': 'A_Pending',
    'A_Pending': 'A_Pending',
    
    # Standalone activities (not clustered, but kept in log as-is following the academic paper)
    'O_Create Offer': 'O_Create Offer',
    'A_Complete': 'A_Complete',
    'W_Handle leads': 'W_Handle leads',
    'A_Concept': 'A_Concept',
}

# Check which activities are not mapped (will be removed)
unmapped = set(df['activity'].unique()) - set(activity_to_cluster.keys())
print(f"Activities not mapped (will be removed): {unmapped}")

Activities not mapped (will be removed): {'W_Assess potential fraud', 'W_Personal Loan collection', 'W_Shortened completion'}


In [10]:
# Filter to keep only activities that are in the clustering mapping
df_filtered = df[df['activity'].isin(activity_to_cluster.keys())].copy()
print(f"Rows after filtering: {len(df_filtered)} (removed {len(df) - len(df_filtered)} rows)")

# Add the clustered activity column
df_filtered['clustered_activity'] = df_filtered['activity'].map(activity_to_cluster)

# Convert timestamps to datetime
df_filtered['Start_Time'] = pd.to_datetime(df_filtered['Start_Time'])
df_filtered['End_Time'] = pd.to_datetime(df_filtered['End_Time'])

# Sort by case_id and Start_Time
df_filtered = df_filtered.sort_values(['case_id', 'Start_Time']).reset_index(drop=True)
print(f"\nUnique clustered activities: {df_filtered['clustered_activity'].nunique()}")
print(df_filtered['clustered_activity'].value_counts())

Rows after filtering: 1198725 (removed 3542 rows)

Unique clustered activities: 11
clustered_activity
W_Complete application     456229
W_Validate application     271617
W_Call incomplete files    191584
A_Create Application        51932
W_Handle leads              47264
O_Create Offer              42995
A_Pending                   34456
A_Concept                   31509
A_Complete                  31362
A_Cancelled                 31329
A_Denied                     8448
Name: count, dtype: int64

Unique clustered activities: 11
clustered_activity
W_Complete application     456229
W_Validate application     271617
W_Call incomplete files    191584
A_Create Application        51932
W_Handle leads              47264
O_Create Offer              42995
A_Pending                   34456
A_Concept                   31509
A_Complete                  31362
A_Cancelled                 31329
A_Denied                     8448
Name: count, dtype: int64


## Aggregate Activities into Clusters

For each cluster occurrence within a case:
- **Start_Time**: First start time of activities in the cluster
- **End_Time**: Last end time of activities in the cluster  
- **Resource**: Resource that executes the first activity in the cluster

In [11]:
# Identify cluster boundaries: a new cluster starts when clustered_activity changes
df_filtered['cluster_change'] = (
    (df_filtered['clustered_activity'] != df_filtered['clustered_activity'].shift()) |
    (df_filtered['case_id'] != df_filtered['case_id'].shift())
)
df_filtered['cluster_id'] = df_filtered['cluster_change'].cumsum()

print(f"Total cluster instances: {df_filtered['cluster_id'].nunique()}")

Total cluster instances: 437370


In [12]:
# Function to get the resource of the first activity in the cluster
def get_first_resource(group):
    return group.sort_values('Start_Time').iloc[0]['resource']

# Aggregate each cluster instance
clustered_log = df_filtered.groupby('cluster_id').agg(
    case_id=('case_id', 'first'),
    activity=('clustered_activity', 'first'),
    Start_Time=('Start_Time', 'min'),
    End_Time=('End_Time', 'max'),
    resource=('resource', lambda x: df_filtered.loc[x.index].sort_values('Start_Time').iloc[0]['resource']),
    # Preserve case attributes
    case_LoanGoal=('case_LoanGoal', 'first'),
    case_ApplicationType=('case_ApplicationType', 'first'),
    case_RequestedAmount=('case_RequestedAmount', 'first'),
).reset_index(drop=True)

print(f"Clustered log shape: {clustered_log.shape}")
clustered_log.head(10)

Clustered log shape: (437370, 8)


,case_id,activity,Start_Time,End_Time,resource,case_LoanGoal,case_ApplicationType,case_RequestedAmount
0,Application_1000086665,A_Create Application,2016-08-03 17:56:59.224,2016-08-03 17:57:21.734,User_1,"Other, see explanation",New credit,5000.0
1,Application_1000086665,W_Handle leads,2016-08-03 17:57:21.734,2016-08-03 17:58:28.286,User_1,"Other, see explanation",New credit,5000.0
2,Application_1000086665,W_Complete application,2016-08-03 17:58:28.286,2016-08-03 17:58:28.293,User_1,"Other, see explanation",New credit,5000.0
3,Application_1000086665,A_Concept,2016-08-03 17:58:28.293,2016-08-03 17:58:28.299,User_1,"Other, see explanation",New credit,5000.0
4,Application_1000086665,W_Complete application,2016-08-04 15:39:19.888,2016-08-05 15:57:07.419,User_14,"Other, see explanation",New credit,5000.0
5,Application_1000086665,O_Create Offer,2016-08-05 15:57:07.419,2016-08-05 15:59:57.320,User_5,"Other, see explanation",New credit,5000.0
6,Application_1000086665,W_Complete application,2016-08-05 15:59:57.320,2016-08-05 16:01:23.286,User_5,"Other, see explanation",New credit,5000.0
7,Application_1000086665,A_Complete,2016-08-05 16:01:23.286,2016-08-05 16:01:23.288,User_5,"Other, see explanation",New credit,5000.0
8,Application_1000086665,W_Complete application,2016-08-05 16:01:23.288,2016-08-09 20:25:32.629,User_5,"Other, see explanation",New credit,5000.0
9,Application_1000086665,A_Cancelled,2016-09-05 08:00:36.107,2016-09-05 08:00:36.829,User_1,"Other, see explanation",New credit,5000.0


In [13]:
# Verify the clustering
print("Activity distribution in clustered log:")
print(clustered_log['activity'].value_counts())
print(f"\nNumber of cases: {clustered_log['case_id'].nunique()}")
print(f"Total events: {len(clustered_log)}")

Activity distribution in clustered log:
activity
W_Complete application     145710
W_Validate application      58342
O_Create Offer              42960
W_Call incomplete files     34819
A_Create Application        31979
A_Concept                   31509
A_Complete                  31362
W_Handle leads              21811
A_Pending                   18361
A_Cancelled                 16613
A_Denied                     3904
Name: count, dtype: int64

Number of cases: 31509
Total events: 437370


In [14]:
# Sort by case_id and Start_Time
clustered_log = clustered_log.sort_values(['case_id', 'Start_Time']).reset_index(drop=True)

# Save the clustered log
clustered_log.to_csv('clustered_log.csv', index=False)
print("Clustered log saved to 'clustered_log.csv'")

# Display sample traces
print("\nSample trace for first case:")
first_case = clustered_log[clustered_log['case_id'] == clustered_log['case_id'].iloc[0]]
print(first_case[['case_id', 'activity', 'resource', 'Start_Time', 'End_Time']])

Clustered log saved to 'clustered_log.csv'

Sample trace for first case:
                   case_id                activity resource  \
0   Application_1000086665    A_Create Application   User_1   
1   Application_1000086665          W_Handle leads   User_1   
2   Application_1000086665  W_Complete application   User_1   
3   Application_1000086665               A_Concept   User_1   
4   Application_1000086665  W_Complete application  User_14   
5   Application_1000086665          O_Create Offer   User_5   
6   Application_1000086665  W_Complete application   User_5   
7   Application_1000086665              A_Complete   User_5   
8   Application_1000086665  W_Complete application   User_5   
9   Application_1000086665             A_Cancelled   User_1   
10  Application_1000086665  W_Complete application   User_1   

                Start_Time                End_Time  
0  2016-08-03 17:56:59.224 2016-08-03 17:57:21.734  
1  2016-08-03 17:57:21.734 2016-08-03 17:58:28.286  
2  2016-08-